In [1]:
# capture image
import time
import cv2
import os

green = (0,255,0)
red = (0,0,255)
white = (255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX
camera = cv2.VideoCapture(0)

width = 1280
height= 960
camera.set(3,width)
camera.set(4,height)

#########################################################################

starting_time = time.time()
Frame_Counter = 0
Cap_frame = 0
Dir_name = "/home/srikanth/Desktop/Arihant/"
number_image_captured = 1
capture_image = False

#########################################################################

while True:
    IsDirExist = os.path.exists(Dir_name)    # check if directory exists or not
#     print(IsDirExist)
        
    if not IsDirExist: os.mkdir(Dir_name)    # if doesn't, create it        

    #####################################################################
        
    Frame_Counter+=1
    ret, frame = camera.read()               # read frames from camera    
    ret, saving_frame = camera.read()        # read saving frame seperatly     
    height, width, dim = saving_frame.shape  # find image width and height
    # print(width, height, dim)
    
    #####################################################################

#     cv2.putText(saving_frame,f"Height: {height}",(30,50),fonts,0.4,(0,255,0),1)
#     cv2.putText(saving_frame,f"Width:  {width}",(30,70),fonts,0.4,(0,255,0),1)

    #####################################################################
    
# capture frame if capture_image is true and number_image_captured < 10
    if capture_image==True and Cap_frame < number_image_captured:
        Cap_frame+=1
        cv2.putText(frame,'Capturing',(50,70),fonts,2,green,1)
        cv2.imwrite(f"{Dir_name}/frame-{Cap_frame}.png", saving_frame)
    else:
        cv2.putText(frame,'Not Capturing',(50,70),fonts,2,red,1)
        Cap_frame = 0
        capture_image =False
    cv2.imshow("frame", frame)
    cv2.imshow("saving Image", saving_frame)
    total_time = time.time()
    frame_time = total_time - starting_time

    #####################################################################
    
    fps = Frame_Counter/frame_time
    # print(fps)
    # print(capture_image)
    
    #####################################################################

#     if cv2.waitKey(1) == ord('q'): break
    
    if cv2.waitKey(1)==27: break    # esc

    if cv2.waitKey(1)==ord('c'): capture_image = True    # capture image        
        
#########################################################################

camera.release()
cv2.destroyAllWindows()

In [13]:
# measure distance
import cv2

green = (0,255,0)
red = (0,0,255)
white = (255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX
cap = cv2.VideoCapture(0)

width = 1280
height= 960
cap.set(3,width)
cap.set(4,height)

#########################################################################

KNOWN_DISTANCE = 160    # distance from camera to object(face) in centimeter
KNOWN_WIDTH = 4.3     # width of face in the real world in centimeter

#########################################################################

# face detector object
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#########################################################################

# focal length finder function
def focal_length(measured_distance, real_width, width_in_rf_image):    
    focal_length_value = (width_in_rf_image * measured_distance) / real_width
    return focal_length_value

#########################################################################

# distance estimation function
def distance_finder(focal_length, real_face_width, face_width_in_frame):    
    distance = (real_face_width * focal_length) / face_width_in_frame
    return distance

#########################################################################

# face detector function
def face_data(image):
    face_width = 0
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_image, 1.3, 5)
    for (x,y,h,w) in faces:
        cv2.rectangle(image,(x,y),(x+w, y+h),white,1)
        face_width = w

    return face_width

#########################################################################

# reading reference image from directory
ref_image = cv2.imread("frame-1.png")
ref_image_face_width = face_data(ref_image)
focal_length_found = focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, ref_image_face_width)
# print(focal_length_found)
# cv2.imshow("ref_image", ref_image)

#########################################################################

while True:
    _, frame = cap.read()
    face_width_in_frame = face_data(frame)
    if face_width_in_frame != 0:
        Distance = distance_finder(focal_length_found, KNOWN_WIDTH, face_width_in_frame)
        cv2.putText(frame, f"Distance = {round(Distance,2)} cm",(50,50),fonts,1,white,2)
    cv2.imshow("frame", frame)

    if cv2.waitKey(1)==27: break    # esc
        
#########################################################################
        
cap.release()
cv2.destroyAllWindows()